In [1]:
print('hello world')

hello world


In [66]:
import numpy as np
import pandas as pd

In [67]:
data = pd.read_csv('mnist_train.csv/mnist_train.csv')

In [68]:
data.shape
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
data = np.array(data)

In [70]:
data.shape

(60000, 785)

In [71]:
print(data[:5, :])

[[5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]]


In [72]:
data = data.T

In [73]:
print(data[:5, :])

[[5 0 4 ... 5 6 8]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [74]:
X = data[1:, :].astype(np.float32) / 255.0
y = data[0].astype(np.int64)

In [75]:
print(y_train[:5])

[5 0 4 1 9]


In [76]:
print(x_train[:, :])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [77]:
# Ensure shapes are (features, samples) for X and (classes, samples) for one-hot labels
# Current layout after transpose: data shape is (785, 60000)
# X will be (784, 60000), y will be (60000,)
print('X shape:', X.shape)
print('y shape:', y.shape)


X shape: (784, 60000)
y shape: (60000,)


In [78]:
def init_params():
    w1 = np.random.randn(10, 784)
    b1 = np.random.randn(10, 1)
    w2 = np.random.randn(10, 10)
    b2 = np.random.randn(10,1)
    return w1, b1, w2, b2

In [79]:
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0, x)

def softmax(x: np.ndarray) -> np.ndarray:
    x_shifted = x - np.max(x, axis=0, keepdims=True)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)

In [80]:
def forward_prop(w1: np.ndarray, b1: np.ndarray, w2: np.ndarray, b2: np.ndarray, X: np.ndarray):
    z1 = w1 @ X + b1
    a1 = relu(z1)
    z2 = w2 @ a1 + b2
    a2 = softmax(z2)
    cache = {"X": X, "z1": z1, "a1": a1, "z2": z2}
    return a2, cache

In [82]:
def one_hot(y: np.ndarray, num_classes: int = 10) -> np.ndarray:
    y = y.astype(int)
    one_hot_y = np.zeros((num_classes, y.size), dtype=np.float32)
    one_hot_y[y, np.arange(y.size)] = 1.0
    return one_hot_y

def backward_prop(a2: np.ndarray, cache: dict, w1: np.ndarray, w2: np.ndarray, b1: np.ndarray, b2: np.ndarray, X: np.ndarray, y: np.ndarray):
    m = y.size
    one_hot_y = one_hot(y, num_classes=a2.shape[0])
    a1 = cache["a1"]
    z1 = cache["z1"]

    dz2 = a2 - one_hot_y
    dw2 = (1.0 / m) * (dz2 @ a1.T)
    db2 = (1.0 / m) * np.sum(dz2, axis=1, keepdims=True)

    relu_grad = (z1 > 0).astype(a1.dtype)
    dz1 = (w2.T @ dz2) * relu_grad
    dw1 = (1.0 / m) * (dz1 @ X.T)
    db1 = (1.0 / m) * np.sum(dz1, axis=1, keepdims=True)

    return dw1, db1, dw2, db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate):
    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    return w1, b1, w2, b2

In [83]:
def accuracy(w1, b1, w2, b2, X, y):
    a2, _ = forward_prop(w1, b1, w2, b2, X)
    y_pred = np.argmax(a2, axis=0)
    return float(np.sum(y_pred == y)) / y.size


def gradient_descent(w1, b1, w2, b2, X, y, learning_rate, num_iterations):
    for i in range(num_iterations):
        a2, cache = forward_prop(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = backward_prop(a2, cache, w1, w2, b1, b2, X, y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, learning_rate)
        if i % 100 == 0:
            print(f"Iteration {i} accuracy: {accuracy(w1, b1, w2, b2, X, y):.4f}")
    return w1, b1, w2, b2

In [84]:
w1, b1, w2, b2 = init_params()
w1, b1, w2, b2 = gradient_descent(w1, b1, w2, b2, X, y, learning_rate=0.1, num_iterations=1000)

Iteration 0 accuracy: 0.1733
Iteration 100 accuracy: 0.3870
Iteration 200 accuracy: 0.4428
Iteration 300 accuracy: 0.4890


KeyboardInterrupt: 